# Documentación: Google Lighthouse Assistant

## Índice
1. Descripción General del Proyecto
2. Archivo de Configuración (.env)
3. Módulo Core - model.py
4. Módulo Core - prompts.py
5. Módulo UI - layout.py
6. Módulo UI - chat.py
7. Archivo Principal - main.py

---

## 1. Descripción general del proyecto

### Estructura del proyecto

El proyecto **Google Lighthouse Assistant** es una aplicación web basada en Streamlit que proporciona un asistente de IA especializado en análisis y optimización de reportes de Google Lighthouse.

### Tecnologías utilizadas

- **Framework UI**: Streamlit
- **LLM Provider**: Groq (llama-3.3-70b-versatile)
- **Lenguaje**: Python 3.13+
- **Gestor de Paquetes**: uv (UV)

### Directorios principales

In [ ]:
"""
lighthouse-assistant/
├── .venv/                 # Entorno virtual de Python
├── app/                   # Aplicación principal
│   ├── core/              # Lógica central
│   │   ├── model.py       # Interfaz con el LLM (Groq)
│   │   ├── prompts.py     # Definición de prompts del sistema
│   │   └── __pycache__/   # Cache de Python
│   ├── ui/                # Interfaz de usuario
│   │   ├── layout.py      # Configuración del layout
│   │   ├── chat.py        # Lógica del chat
│   │   └── __pycache__/   # Cache de Python
│   ├── main.py            # Punto de entrada de la aplicación
│   └── .env               # Variables de entorno
│── docs/                  # Documentación
│── .gitignore             # Archivos y directorios ignorados por Git
│── .python-version        # Versión de Python utilizada
│── .pyprec.toml           # Configuración de pyproject.toml
│── README.md              # Documentación del proyecto
└── uv.lock                # Archivo de bloqueo de dependencias de uv
"""

## 2. Archivo de Configuración (.env)

- El archivo `.env` almacena **variables de entorno sensibles** que no deben ser versionadas en git.
- Contiene configuraciones como claves API, tokens y otros datos privados.

### Variables

- `GROQ_API_KEY`: Clave API para acceder a los modelos de Groq

In [ ]:
# Ejemplo de cómo se carga .env en main.py
import os
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la clave API
groq_api_key = os.getenv("GROQ_API_KEY")

## 3. Módulo Core

### `model.py`

- Este módulo es la **interfaz entre la aplicación y el LLM (Groq)**.
- Gestiona la comunicación con el modelo de IA y procesa las respuestas.

In [ ]:
import os
from groq import Groq
from .prompts import SYSTEM_PROMPT

def get_model_response(messages: list[dict]) -> str:
    """
    Obtiene una respuesta del modelo LLM basada en los mensajes del usuario.
    
    Args:
        messages: Lista de diccionarios con role y content
        
    Returns:
        str: Respuesta del modelo o mensaje de error
    """
    try:
        # 1. Crear cliente de Groq con la API Key
        client = Groq(api_key=os.getenv("GROQ_API_KEY"))
        
        # 2. Crear mensaje del sistema con el system prompt
        system_message = {"role": "system", "content": "Tu eres Google Lighthouse Assistant..."}
        
        # 3. Combinar system message + mensajes del usuario
        all_messages = [system_message] + messages
        
        # 4. Hacer request al API de Groq
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=all_messages,          # ← Incluye el system prompt
            temperature=0.7,                # Creatividad de respuestas
            max_tokens=2000,                # Límite de tokens
            top_p=1,                        # Muestreo nucleus
            stream=False,                   # Respuesta completa, no en streaming
        )
        
        # 5. Extraer y retornar el contenido de la respuesta
        return response.choices[0].message.content
        
    except Exception as e:
        # Manejo de errores
        return "Lo siento, ha ocurrido un error al procesar tu solicitud."

# Ejemplo de uso
ejemplo_mensajes = [
    {"role": "user", "content": "¿Cómo puedo mejorar el rendimiento de mi sitio web?"}
]

# Esta función es llamada desde ui/chat.py

### `prompts.py`

- Este módulo contiene **todas las instrucciones y definiciones de prompts** para el asistente.
- Define qué puede hacer el LLM, sus restricciones y cómo responder.

In [ ]:
# SYSTEM_PROMPT
SYSTEM_PROMPT = """Eres Google Lighthouse Assistant, un asistente experto especializado en análisis y optimización web basado en Google Lighthouse.

## IDENTIDAD Y PROPÓSITO
Eres un asistente técnico profesional que ayuda a desarrolladores, diseñadores y profesionales web a:
- Analizar y mejorar el rendimiento de sus sitios web
- Cumplir con estándares de accesibilidad
- Implementar mejores prácticas web
- Optimizar para SEO
- Desarrollar Progressive Web Apps (PWAs)

## RESTRICCIONES Y LÍMITES
NO puedes ayudar con:
- Temas no relacionados con web performance, accesibilidad, mejores prácticas, SEO o PWA
- Análisis de seguridad más allá de HTTPS
- Hosting o infraestructura en general
- Bases de datos o backend (excepto impacto en performance)
- Marketing digital (excepto SEO técnico)
"""

# Resultado: El LLM SOLO responderá preguntas sobre los 5 temas permitidos

In [ ]:
# Áreas de Expertise - Performance
PERFORMANCE_EXPERTISE = """
**a) Análisis de Rendimiento Web (Performance)**
- Métricas Core Web Vitals (LCP, FID, CLS)
- Tiempo de carga y optimización
- Minificación de recursos
- Compresión de imágenes
- Lazy loading
- Code splitting
- Caché de navegador
- CDNs
- Minimización de JavaScript/CSS
"""

# Áreas de Expertise - Accessibility
ACCESSIBILITY_EXPERTISE = """
**b) Accesibilidad (Accessibility)**
- WCAG 2.1 (A, AA, AAA)
- Contraste de colores
- Navegación por teclado
- ARIA labels y roles
- Texto alternativo para imágenes
- Estructura semántica HTML
- Readers de pantalla
- Focus management
"""

# Áreas de Expertise - Best Practices
BEST_PRACTICES_EXPERTISE = """
**c) Mejores Prácticas (Best Practices)**
- Seguridad HTTPS
- Validación de HTML/CSS
- Gestión de errores
- Logs de consola
- Frameworks modernos
- Patrones de desarrollo
- Estructura de código
- Documentación
"""

# Áreas de Expertise - SEO
SEO_EXPERTISE = """
**d) SEO (Search Engine Optimization)**
- Meta tags
- Open Graph
- Structured data (Schema.org)
- Sitemaps
- Robots.txt
- Mobile-friendly design
- Velocidad de página
- Canonical URLs
- Heading hierarchy
"""

# Áreas de Expertise - PWA
PWA_EXPERTISE = """
**e) Progressive Web Apps (PWA)**
- Service Workers
- Manifest.json
- Offline functionality
- Push notifications
- App Shell architecture
- Instalabilidad
- Responsive design
"""

In [ ]:
# ANALYSIS_PROMPT
ANALYSIS_PROMPT = """Analiza el siguiente reporte de Google Lighthouse y proporciona:

1. **Resumen General**
   - Puntuación global
   - Categorías principales
   - Estado general del sitio

2. **Análisis Detallado por Categoría**
   - Rendimiento (Performance)
   - Accesibilidad (Accessibility)
   - Mejores Prácticas (Best Practices)
   - SEO
   - PWA (si aplica)

3. **Problemas Críticos**
   - Top 5 issues más impactantes
   - Severidad y urgencia
   - Impacto estimado

4. **Plan de Acción**
   - Acciones inmediatas (semana 1)
   - Mejoras a corto plazo (mes 1)
   - Optimizaciones a largo plazo

5. **Recursos y Documentación**
   - Links relevantes de Google Lighthouse
   - Documentación técnica
   - Herramientas recomendadas

Reporte JSON:
{report}"""

# Caso de uso: Cuando el usuario carga un archivo JSON de Lighthouse
# El prompt se formatea con el contenido del reporte

In [ ]:
# BEST_PRACTICES_PROMPT
BEST_PRACTICES_PROMPT = """Eres un experto en optimización web. Responde la siguiente pregunta relacionada con:
- Análisis de rendimiento web (Performance)
- Accesibilidad (Accessibility)
- Mejores prácticas (Best Practices)
- SEO (Search Engine Optimization)
- Progressive Web Apps (PWA)

Si la pregunta no está relacionada con estos temas, indica amablemente que no puedes ayudar.

Pregunta: {question}"""

# Caso de uso: Cuando el usuario hace una pregunta general

In [ ]:
# SCOPE_VALIDATION_PROMPT
SCOPE_VALIDATION_PROMPT = """Evalúa si la siguiente pregunta está relacionada con los temas permitidos:
- Análisis de rendimiento web (Performance)
- Accesibilidad (Accessibility)
- Mejores prácticas (Best Practices)
- SEO (Search Engine Optimization)
- Progressive Web Apps (PWA)

Pregunta: {question}

Responde SOLO con:
- "IN_SCOPE" si la pregunta está relacionada
- "OUT_OF_SCOPE" si no está relacionada

Explicación breve de tu decisión (una línea)."""

# Caso de uso: Validar si una pregunta debe ser procesada o rechazada

In [ ]:
# TECHNICAL_TERMS
TECHNICAL_TERMS = {
    "Core Web Vitals": "Métricas clave de Google que miden la experiencia del usuario: LCP, FID/INP y CLS",
    "LCP": "Largest Contentful Paint - Tiempo hasta que se pinta el contenido más grande",
    "FID": "First Input Delay - Latencia de la primera interacción del usuario",
    "INP": "Interaction to Next Paint - Nueva métrica que reemplaza FID",
    "CLS": "Cumulative Layout Shift - Cambios inesperados en el layout durante la carga",
    "WCAG": "Web Content Accessibility Guidelines - Estándares internacionales de accesibilidad",
    "Service Worker": "Script que se ejecuta en background para funcionalidad offline",
    "Manifest": "Archivo JSON que define propiedades de una PWA",
    "SEO Técnico": "Optimización del sitio para que search engines lo rastreen e indexen",
    "Schema.org": "Vocabulario para structured data que ayuda a search engines a entender el contenido",
}

# Uso: retorna todo el diccionario para consultar definiciones rápidas de términos clave

In [ ]:
# LIGHTHOUSE_CATEGORIES - Performance
PERFORMANCE_CATEGORY = {
    "performance": {
        "description": "Rendimiento de carga y ejecución del sitio",
        "key_metrics": [
            "First Contentful Paint (FCP)",
            "Largest Contentful Paint (LCP)",
            "Cumulative Layout Shift (CLS)",
            "Time to Interactive (TTI)",
            "Total Blocking Time (TBT)"
        ],
        "improvements": [
            "Optimizar imágenes",
            "Minificar CSS/JavaScript",
            "Implementar lazy loading",
            "Usar CDN",
            "Reducir JavaScript no usado"
        ]
    }
}

# LIGHTHOUSE_CATEGORIES - Accessibility
ACCESSIBILITY_CATEGORY = {
    "accessibility": {
        "description": "Accesibilidad para usuarios con discapacidades",
        "key_metrics": [
            "Contraste de color",
            "Navegación por teclado",
            "ARIA attributes",
            "Labels de formularios",
            "Estructura semántica"
        ],
        "improvements": [
            "Mejorar contraste de colores",
            "Agregar ARIA labels",
            "Asegurar navegación por teclado",
            "Añadir alt text a imágenes",
            "Usar headings semánticamente"
        ]
    }
}

# LIGHTHOUSE_CATEGORIES - Best Practices
BEST_PRACTICES_CATEGORY = {
    "best-practices": {
        "description": "Mejores prácticas de desarrollo web moderno",
        "key_metrics": [
            "HTTPS habilitado",
            "No errores de console",
            "Frameworks modernos",
            "Versionamiento correcto",
            "Permisos de usuario apropiados"
        ],
        "improvements": [
            "Implementar HTTPS",
            "Solucionar errores de console",
            "Actualizar dependencias",
            "Usar semántica HTML correcta",
            "Implementar error handling"
        ]
    }
}

In [ ]:
# LIGHTHOUSE_CATEGORIES - SEO
SEO_CATEGORY = {
    "seo": {
        "description": "Optimización para motores de búsqueda",
        "key_metrics": [
            "Meta tags",
            "Viewport meta",
            "Structured data",
            "Mobile friendly",
            "Legibilidad"
        ],
        "improvements": [
            "Agregar meta descriptions",
            "Implementar structured data",
            "Mejorar heading hierarchy",
            "Optimizar para mobile",
            "Crear sitemap XML"
        ]
    }
}

# LIGHTHOUSE_CATEGORIES - PWA
PWA_CATEGORY = {
    "pwa": {
        "description": "Progressive Web App capabilities",
        "key_metrics": [
            "Installable",
            "Service Worker",
            "Manifest present",
            "Offline support",
            "Theme color"
        ],
        "improvements": [
            "Crear manifest.json",
            "Implementar Service Worker",
            "Hacer app instalable",
            "Soportar modo offline",
            "Agregar splash screens"
        ]
    }
}

# Cada categoría contiene:
# - description: Explicación de qué mide
# - key_metrics: Métricas principales a considerar
# - improvements: Sugerencias de mejora ordenadas por impacto

# 5. Módulo UI

##  `layout.py`

- Define la **estructura visual de la aplicación Streamlit**.
- Configura el layout, título, widgets y componentes de la interfaz.

In [ ]:
import streamlit as st


def render_layout():
    """
    Configura y renderiza el layout principal de la aplicación.
    Se ejecuta al iniciar la app.
    """
    
    # 1. Configurar página
    st.set_page_config(
        page_title="Google Lighthouse Assistant",  # Título en pestaña del navegador
        layout="wide"                              # Layout ancho (2 columnas)
    )
    
    # 2. Título principal
    st.title("Asistente :blue[Google Lighthouse]")  # :blue[] aplica color
    
    # 3. Barra lateral (sidebar)
    with st.sidebar:
        st.file_uploader(
            "Carga tus reportes JSON",
            type="json",                    # Solo acepta archivos .json
            accept_multiple_files=True      # Permite múltiples archivos
        )

# Nota: Esta función es llamada desde main.py

## `chat.py`

- Maneja **la lógica del chat interactivo**.
- Gestiona la visualización de mensajes, entrada del usuario y respuestas del modelo.

In [ ]:
import streamlit as st
from core.model import get_model_response


def render_chat():
    """
    Renderiza la interfaz de chat y maneja las interacciones del usuario.
    """
    
    # 1. Inicializar historial de mensajes en session_state (se mantiene entre recargas de página)
    if "messages" not in st.session_state:
        st.session_state.messages = []
    
    # 2. Mostrar todos los mensajes previos
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
    
    # 3. Input del usuario
    if prompt := st.chat_input():
        
        # 3a. Mostrar mensaje del usuario
        with st.chat_message("user"):
            st.markdown(prompt)
        
        # 3b. Guardar en historial
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        # 3c. Obtener respuesta del modelo
        with st.chat_message("assistant"):
            with st.spinner("Pensando..."):  # Indicador de carga
                response = get_model_response(st.session_state.messages)
                st.markdown(response)
        
        # 3d. Guardar respuesta en historial
        st.session_state.messages.append({"role": "assistant", "content": response})


# Nota: Esta función es llamada desde main.py después de render_layout()

# 7. Archivo Principal - `main.py`

- Es el **punto de entrada de la aplicación**.
- Orquesta la inicialización de todos los módulos y el flujo principal.

In [ ]:
from dotenv import load_dotenv

from ui.layout import render_layout
from ui.chat import render_chat

# 1. Cargar variables de entorno desde .env
load_dotenv()


def main():
    """
    Función principal que inicializa y orquesta la aplicación.
    
    Flujo:
    1. Carga configuración del .env
    2. Renderiza layout base
    3. Renderiza interfaz de chat
    """
    
    # 2. Renderizar layout (configuración de página, sidebar)
    render_layout()
    
    # 3. Renderizar chat (interfaz de conversación)
    render_chat()


# Punto de entrada
if __name__ == "__main__":
    main()


# Para ejecutar la aplicación:
# streamlit run app/main.py

# 📊 Diagrama de Flujo Completo

```
┌─────────────────────────────────────────────────────────────────┐
│                    USUARIO INICIA APP                           │
│              streamlit run app/main.py                          │
└────────────────────────────┬────────────────────────────────────┘
                             │
                             ↓
            ┌────────────────────────────────┐
            │       main.py                  │
            │  1. load_dotenv()              │
            │  2. render_layout()            │
            │  3. render_chat()              │
            └────────┬───────────┬───────────┘
                     │           │
        ┌────────────▼─┐    ┌────▼──────────┐
        │ layout.py    │    │  chat.py      │
        │ ──────────── │    │ ──────────────│
        │ • Configura  │    │ • Historial   │
        │   página     │    │ • Input user  │
        │ • Sidebar    │    │ • Output resp │
        └────────┬────-┘    └────┬──────────┘
                 │               │
                 │          ┌────▼──────────────┐
                 │          │ model.py          │
                 │          │ ──────────────────│
                 │          │ get_model_response│
                 │          └────┬──────────────┘
                 │               │
                 │          ┌────▼──────────────┐
                 │          │   Groq API        │
                 │          │ ──────────────────│
                 │          │ llama-3.3-70b     │
                 │          └────┬──────────────┘
                 │               │
                 │          ┌────▼──────────────┐
                 │          │ prompts.py        │
                 │          │ ──────────────────│
                 │          │ SYSTEM_PROMPT     │
                 │          │ (restricciones)   │
                 │          └────┬──────────────┘
                 │               │
                 └───────────-───┴──→ Respuesta al usuario
```

## Cómo Ejecutar

```bash
# 1. Activar entorno virtual
source .venv/bin/activate

# 2. Instalar dependencias
uv pip install -r requirements.txt

# 3. Configurar .env con tu GROQ_API_KEY

# 4. Ejecutar la aplicación
streamlit run app/main.py
```

## Dependencias Principales

- **streamlit**: Framework de UI
- **groq**: Cliente para Groq API
- **python-dotenv**: Carga variables de entorno
- **python**: 3.13+